In [1]:
import torch 
import time 
from sklearn.model_selection import KFold

import torch.nn as nn
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter
from gensim.models.phrases import Phrases
from gensim.models import Word2Vec
from src.train_utils import set_seed, ModelSave, get_torch_device, EarlyStop, TrainParams
from src.metric import  multi_cls_metrics,multi_cls_log
from src.dataset.tokenizer import GensimTokenizer
from src.loss import BootstrapCrossEntropy

from iflytek_app.dataset import MixDataset
from iflytek_app.process import train_process, test_process, result_process,kfold_inference
from iflytek_app.models import TextcnnPseudoLabel, Textcnn
device = get_torch_device()
set_seed()

No GPU available, using the CPU instead.


In [2]:
c2v = GensimTokenizer( Word2Vec.load('./checkpoint/char_min1_win5_sg_d100'))
w2v = GensimTokenizer(Word2Vec.load('./checkpoint/phrase_min1_win5_sg_d100'))
w2v.init_vocab()
c2v.init_vocab()
phraser = Phrases.load('./checkpoint/phrase_tokenizer')
df, label2idx = train_process()
test = test_process()
label2idx.update({'unlabel':-1})


                id           l1           l2          len
count  4199.000000  4199.000000  4199.000000  4199.000000
mean   2099.000000     8.969278    37.087878    46.057156
std    1212.291219     4.576621    79.204914    79.332999
min       0.000000     2.000000     1.000000     4.000000
25%    1049.500000     5.000000     6.000000    15.000000
50%    2099.000000     8.000000    12.000000    22.000000
75%    3148.500000    12.000000    26.000000    36.000000
max    4198.000000    32.000000   946.000000   961.000000
{'14784131 14858934 14784131 14845064': 0, '14852788 14717848 15639958 15632020': 1, '14844856 14724258 14925237 14854807': 2, '14925756 15639967 14853254 14728639': 3, '14844593 14924945': 4, '15709098 14716590 14924703 14779559': 5, '14726332 14728344 14854542 14844591': 6, '14858934 15636660 15704193 14849963': 7, '15710359 14847407 14845602 14859696': 8, '14794687 14782344': 9, '15630486 15702410 14718849 15709093': 10, '15632285 15706536 14721977 14925219': 11, '147829

In [8]:
log_steps = 10
save_steps = 20
kfold=5
tp = TrainParams(
    epoch_size=30,
    lr=1e-3,
    loss_fn=nn.CrossEntropyLoss(),
    max_seq_len=1000,
    batch_size=64,
    dropout_rate=0.5,
    label_size = len(label2idx),
    vocab_size = w2v.vocab_size,
    embedding_dim = w2v.embedding_size + c2v.embedding_size,
    embedding1=c2v.embedding, 
    embedding2 =w2v.embedding,
    filter_size=100,
    kernel_size_list = [2,3,4,5],
    hidden_size = 200,
    early_stop_params = {
        'monitor':'f1_micro',
        'mode':'max',
        'min_delta': 0,
        'patience':5,
        'verbose':False
    },
    scheduler_params={'mode': 'max',
                     'factor': 0.3,
                     'patience': 1,
                     'verbose': True,
                     'threshold':0.0001,
                     'threshold_mode':'rel',
                     'cooldown':0,
                     'min_lr':1e-6},
    num_train_steps = int(df.shape[0]/kfold *(kfold-1)),
    T1=5,
    T2=20,
    alpha_f=3,
    q=0.7
    
)

In [9]:
kf = KFold(n_splits=kfold, shuffle=True, random_state=24)
for fold,(train_index, valid_index) in enumerate(kf.split(df)):
    train, valid = df.iloc[train_index], df.iloc[valid_index]

    train, valid = df.iloc[train_index], df.iloc[valid_index]

    # combine label and unlabel data
    train_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx, 
                               train['name'].values.tolist(),train['description'].values.tolist() ,
                               train['label'].values.tolist())
    unlabel_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx,
                                 test['name'].values, test['description'].values, ['unlabel'] *test.shape[0])
    valid_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx, 
                               valid['name'].values, valid['description'].values, valid['label'].values)
    train_sampler = RandomSampler(train_dataset)
    unlabel_sampler = RandomSampler(unlabel_dataset)
    valid_sampler = SequentialSampler(valid_dataset)
    train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=tp.batch_size)
    valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=tp.batch_size)
    unlabel_loader = DataLoader(unlabel_dataset, sampler=unlabel_sampler, batch_size=tp.batch_size)
    
    tp.update({'num_train_steps': len(train_dataset)})
    
    CKPT = './checkpoint/textcnn_pseudo_label/k{}'.format(fold)
    saver = ModelSave(CKPT, continue_train=False)
    saver.init()
    tb = SummaryWriter(CKPT)
    es = EarlyStop(**tp.early_stop_params)
    global_step = 0
    model = TextcnnPseudoLabel(tp)
    optimizer, scheduler = model.get_optimizer()
    
    for epoch_i in range(tp['epoch_size']):
        if global_step==1:
            print(model)
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10}  | {'Elapsed':^9}")
        print("-"*60)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()
        total_loss, batch_loss, batch_counts = 0, 0, 0

        model.train()
        unlabel_iter = iter(unlabel_loader)
        for step, batch in enumerate(train_loader):
            global_step +=1
            batch_counts +=1

            try:
                unlabel_batch = next(unlabel_iter)
            except:
                unlabel_iter = iter(unlabel_loader)
                unlabel_batch = next(unlabel_iter)
                
            #Forward propogate
            model.zero_grad()
            feature = {k:torch.cat([v,unlabel_batch[k]],dim=0).to(device) for k,v in batch.items()}
            logits = model(feature)
            loss= model.compute_loss(feature, logits)
            tb.add_scalar('loss/sup_loss', model.supervised_loss, global_step=global_step)
            tb.add_scalar('loss/unsup_loss', model.unsupervised_loss, global_step=global_step)
            tb.add_scalar('loss/avg_loss', loss, global_step=global_step)
            batch_loss += loss.item()
            total_loss += loss.item()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()

            # Log steps for train loss logging
            if (step % log_steps == 0 and step != 0) or (step == len(train_loader) - 1):
                time_elapsed = time.time() - t0_batch
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^9} | {time_elapsed:^9.2f}")
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

            # Save steps for ckpt saving and dev evaluation
            if (step % save_steps == 0 and step != 0) or (step == len(train_loader) - 1):
                val_metrics = multi_cls_metrics(model, valid_loader, device)
                for key, val in val_metrics.items():
                    tb.add_scalar(f'metric/{key}', val, global_step=global_step)
                avg_train_loss = total_loss / step
                tb.add_scalars('loss/train_valid',{'train': avg_train_loss,
                                                    'valid': val_metrics['val_loss']}, global_step=global_step)
                saver(total_loss / step, val_metrics['val_loss'], epoch_i, global_step, model, optimizer, scheduler)
        model.epoch_update()
        # On Epoch End: calcualte train & valid loss and log overall metrics
        time_elapsed = time.time() - t0_epoch
        val_metrics = multi_cls_metrics(model, valid_loader, device)
        avg_train_loss = total_loss / step
        scheduler.step(val_metrics['f1_micro'])
        print("-"*70)
        print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_metrics['val_loss']:^10.6f} | {time_elapsed:^9.2f}")
        multi_cls_log(epoch_i, val_metrics)
        print("\n")
        if es.check(val_metrics):
            break

./checkpoint/textcnn_pseudo_label/k0 model cleaned
 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   10    |   4.300034   |     -     |   50.15  
   1    |   20    |   2.286821   |     -     |   44.69  
   1    |   30    |   2.509486   |     -     |   52.32  
   1    |   40    |   2.054647   |     -     |   44.52  
   1    |   50    |   1.848502   |     -     |   54.24  
   1    |   52    |   1.885293   |     -     |   7.63   
----------------------------------------------------------------------
   1    |    -    |   2.655106   |    nan     |  263.88  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   1    |  23.051%  |  73.768%  |  34.175%  |     26.647%     |   23.051%    |  45.238%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F

   8    |   20    |   0.402575   |     -     |   44.49  
   8    |   30    |   0.203739   |     -     |   52.34  
   8    |   40    |   0.188643   |     -     |   44.70  
   8    |   50    |   0.185625   |     -     |   55.25  
   8    |   52    |   0.270617   |     -     |   7.82   
----------------------------------------------------------------------
   8    |    -    |   0.288779   |    nan     |  262.75  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   8    |  56.319%  |  85.103%  |  62.082%  |     69.942%     |   56.319%    |  73.929%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
   8    |  73.929%  |     -     |     -     |     73.929%     |   73.929%    |  73.929%  




 Epoch  |  Batch  |  Train Loss  |  Val 

   2    |   40    |   1.521312   |     -     |   42.62  
   2    |   50    |   1.443832   |     -     |   51.91  
   2    |   52    |   1.457856   |     -     |   7.36   
----------------------------------------------------------------------
   2    |    -    |   2.100560   |    nan     |  251.90  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   2    |  40.562%  |  74.733%  |  49.866%  |     41.429%     |   40.562%    |  58.571%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
   2    |  58.571%  |     -     |     -     |     58.571%     |   58.571%    |  58.571%  




 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   3    |   10    |   1.407856   |

  10    |   10    |   0.405720   |     -     |   48.06  
  10    |   20    |   0.451079   |     -     |   43.06  
  10    |   30    |   0.228590   |     -     |   50.14  
  10    |   40    |   0.202712   |     -     |   42.81  
  10    |   50    |   0.190721   |     -     |   51.70  
  10    |   52    |   0.280732   |     -     |   7.41   
----------------------------------------------------------------------
  10    |    -    |   0.302989   |    nan     |  252.67  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
  10    |  64.244%  |  79.621%  |  70.663%  |     65.713%     |   64.244%    |  73.095%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
  10    |  73.095%  |     -     |     -     |     73.095%     |   73.095%    

  17    |   20    |   0.210287   |     -     |   46.98  
  17    |   30    |   0.031494   |     -     |   64.59  
  17    |   40    |   0.024993   |     -     |   50.14  
  17    |   50    |   0.036464   |     -     |   57.53  
  17    |   52    |   0.035572   |     -     |   7.85   
----------------------------------------------------------------------
  17    |    -    |   0.110885   |    nan     |  285.43  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
  17    |  61.665%  |  78.724%  |  71.279%  |     70.654%     |   61.665%    |  75.595%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
  17    |  75.595%  |     -     |     -     |     75.595%     |   75.595%    |  75.595%  




./checkpoint/textcnn_pseudo_label/k2 mod

KeyboardInterrupt: 

## K-Fold Evalutaion

In [21]:
test = test_process()
test_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx,  test['name'].values, test['description'].values)
test_sampler = SequentialSampler(test_datase t)
test_loader = DataLoader(test_dataset, sampler=test_sampler, batch_size=tp.batch_size)

In [25]:
result = kfold_inference(test_loader, tp, Textcnn, './checkpoint/textcnn_pseudo_label', 5, device)
result['pred'] = result['pred_avg']
result_process(result, label2idx, './submit/textcnn_pseudo_label_5fold_avg.csv')